In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import re

# Настройки
MAX_REVIEWS = 1000
MOVIES_TO_COLLECT = 30  # Уменьшили количество фильмов
REVIEWS_PER_MOVIE = 40  # Увеличили отзывов с фильма
THREADS = 5  # Количество потоков

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_movie_links():
    """Быстро собирает ссылки на популярные фильмы"""
    url = "https://www.kinopoisk.ru/lists/movies/popular-films/"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        return [f"https://www.kinopoisk.ru{a['href']}" 
                for a in soup.select('a[href*="/film/"]')[:MOVIES_TO_COLLECT]]
    except:
        return []

def process_movie(movie_url):
    """Обрабатывает один фильм и возвращает его отзывы"""
    reviews = []
    try:
        response = requests.get(f"{movie_url}reviews/", headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        movie_title = soup.find('span', class_='styles_mainTitle__iwQCq').text.strip()
        review_blocks = soup.select('div.reviewItem')[:REVIEWS_PER_MOVIE]
        
        for review in review_blocks:
            try:
                date = review.find('span', class_='_reachbanner_').text.strip()
                date = datetime.strptime(date, '%d.%m.%Y').strftime('%Y-%m-%d') if date else "Неизвестно"
                
                rating = review.find('span', class_='film-rating-value')
                rating = rating.text.strip() if rating else "5"  # По умолчанию средний рейтинг
                
                text = review.find('p').text.strip() if review.find('p') else ""
                
                if text and any(c in text for c in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'):
                    reviews.append({
                        'ID': f"rev_{random.randint(10000, 99999)}",
                        'Атауы': movie_title,
                        'Күні': date,
                        'Баға': str(min(5, max(1, round(float(rating)/2)))),  # Конвертируем 10-балльную в 5-балльную
                        'Пікір': text
                    })
            except:
                continue
    
    except Exception as e:
        print(f"Ошибка с {movie_url}: {str(e)[:50]}...")
    
    return reviews

def collect_reviews_fast():
    """Быстрый сбор отзывов с использованием многопоточности"""
    movie_links = get_movie_links()
    all_reviews = []
    
    with ThreadPoolExecutor(max_workers=THREADS) as executor:
        results = executor.map(process_movie, movie_links)
        for movie_reviews in results:
            all_reviews.extend(movie_reviews)
            print(f"Собрано отзывов: {len(all_reviews)}", end='\r')
            if len(all_reviews) >= MAX_REVIEWS:
                break
    
    return all_reviews[:MAX_REVIEWS]

# Запуск сбора
start_time = time.time()
print("Начал сбор отзывов...")
reviews_data = collect_reviews_fast()

# Сохранение
df = pd.DataFrame(reviews_data)
df.to_csv('fast_kinopoisk_review.csv', index=False, encoding='utf-8-sig')

print(f"\nГотово! Собрано {len(df)} отзывов за {time.time()-start_time:.1f} секунд")
print(df.head())

Начал сбор отзывов...

Готово! Собрано 0 отзывов за 0.6 секунд
Empty DataFrame
Columns: []
Index: []
